In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r"bert_text.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         21000 non-null  int64 
 1   LABEL              21000 non-null  int64 
 2   PRODUCT_TITLE      21000 non-null  object
 3   REVIEW_TITLE       21000 non-null  object
 4   REVIEW_TEXT        21000 non-null  object
 5   Category_Label     21000 non-null  int64 
 6   RATING             21000 non-null  int64 
 7   VERIFIED_PURCHASE  21000 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 1.3+ MB


In [4]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df

,LABEL,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT,Category_Label,RATING,VERIFIED_PURCHASE
0,1,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav...",20,4,0
1,1,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...,29,4,1
2,1,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,2,3,0
3,1,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...,18,4,0
4,1,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...,3,4,0
...,...,...,...,...,...,...,...
20995,0,"Madden Girl Women's Gettaw Pump,Red Patent,7.5...",wide width is great!,"I bought these for work. I have high arches, ...",22,4,1
20996,0,"crocs Unisex Classic Clog,Khaki,6 US Men's / 8...",Love crocs!,Crocs are one of only two brands of shoes that...,22,4,1
20997,0,Minnetonka Men's 703 Leather Laced Softsole Mo...,I love moccasins This fit like it was custom m...,I love moccasins This fit like it was custom ...,22,5,1
20998,0,Ariat Womens Unbridled Fatbaby 9 B Powder Brown,"This fit well, comfortable, best investment",I wish these were a little more durable. I got...,22,5,1


In [6]:
sen_w_feats = []
labels = []
for index, row in df.iterrows():
    combined = ""
    combined += row["PRODUCT_TITLE"]+" "
    combined += row["REVIEW_TITLE"]+" "
    combined += row["REVIEW_TEXT"]+" "
   
    combined += "This item is classified under category label {:}. ".format(row["Category_Label"])
    combined += "I rate this item {:} out of 5 stars. ".format(row["RATING"])
    combined += "It is {:} that I have verified this purchase. ".format("true" if row["VERIFIED_PURCHASE"] else "false")
    sen_w_feats.append(combined)
    labels.append(row["LABEL"])
print('  DONE.')
print('Dataset contains {:,} samples.'.format(len(sen_w_feats)))

  DONE.
Dataset contains 21,000 samples.


In [7]:
import textwrap
wrapper = textwrap.TextWrapper(width=80)
print(wrapper.fill(sen_w_feats[1]))

Note 3 Battery : Stalion Strength Replacement 3200mAh Li-Ion Battery for Samsung
Galaxy Note 3 [24-Month Warranty] with NFC Chip + Google Wallet Capable New era
for batteries Lithium batteries are something new introduced in the market there
average developing cost is relatively high but Stallion doesn't compromise on
quality and provides us with the best at a low cost.<br />There are so many in
built technical assistants that act like a sensor in their particular forté. The
battery keeps my phone charged up and it works at every voltage and a high
voltage is never risked. This item is classified under category label 29. I rate
this item 4 out of 5 stars. It is true that I have verified this purchase.


In [8]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [9]:
!pip install transformers

In [10]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
)

desc = model.cuda()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
batch_size = 32
learning_rate = 1e-5
epochs = 4

In [13]:
max_len = 0
for sent in sen_w_feats:
    input_ids = tokenizer.encode(sent, padding=True, max_length=512, truncation=True,add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)

Max sentence length:  512


In [14]:
input_ids = []
attention_masks = []
print('Encoding all reviews in the dataset...')
for sent in sen_w_feats:
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True, 
                        max_length = max_len,          
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   
                        return_tensors = 'pt', 
                )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
print('DONE.')

Encoding all reviews in the dataset...
DONE.


In [15]:
from torch.utils.data import TensorDataset
import random
import numpy as np
train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))
test_size = len(df) - (train_size + val_size)
indeces = np.arange(0, len(df))
random.shuffle(indeces)

train_idx = indeces[0:train_size]
val_idx = indeces[train_size:(train_size + val_size)]
test_idx = indeces[(train_size + val_size):]

train_dataset = TensorDataset(input_ids[train_idx], attention_masks[train_idx], labels[train_idx])
val_dataset = TensorDataset(input_ids[val_idx], attention_masks[val_idx], labels[val_idx])
test_dataset = TensorDataset(input_ids[test_idx], attention_masks[test_idx], labels[test_idx])

In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size
        )

In [17]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = 1e-8
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
from transformers import get_linear_schedule_with_warmup
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [19]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [20]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [21]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()


for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        result = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    print("")
    print("Running Validation...")

    t0 = time.time()

 
    model.eval()


    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():

            result = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        loss = result.loss
        logits = result.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    525.    Elapsed: 0:01:55.
  Batch    80  of    525.    Elapsed: 0:03:48.
  Batch   120  of    525.    Elapsed: 0:05:42.
  Batch   160  of    525.    Elapsed: 0:07:35.
  Batch   200  of    525.    Elapsed: 0:09:29.
  Batch   240  of    525.    Elapsed: 0:11:23.
  Batch   280  of    525.    Elapsed: 0:13:16.
  Batch   320  of    525.    Elapsed: 0:15:10.
  Batch   360  of    525.    Elapsed: 0:17:04.
  Batch   400  of    525.    Elapsed: 0:18:57.
  Batch   440  of    525.    Elapsed: 0:20:51.
  Batch   480  of    525.    Elapsed: 0:22:45.
  Batch   520  of    525.    Elapsed: 0:24:38.

  Average training loss: 0.43
  Training epcoh took: 0:24:53

Running Validation...
  Accuracy: 0.84
  Validation Loss: 0.40
  Validation took: 0:01:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    525.    Elapsed: 0:01:54.
  Batch    80  of    525.    Elapsed: 0:03:47.
  Batch   120  of    525.    Elapsed: 0:05:41.
  Batch  

In [22]:
import pandas as pd

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.430472,0.401534,0.843087,0:24:53,0:01:01
2,0.344095,0.390799,0.850000,0:24:51,0:01:01
3,0.289558,0.398679,0.858523,0:24:52,0:01:01
4,0.248236,0.411490,0.860417,0:24:52,0:01:01


In [24]:

prediction_sampler = SequentialSampler(test_dataset)
prediction_dataloader = DataLoader(test_dataset, sampler=prediction_sampler, batch_size=batch_size)

print('Predicting labels for {:,} test sentences...'.format(len(test_dataset)))


model.eval()
predictions , true_labels = [], []
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      result = model(b_input_ids,
                     token_type_ids=None,
                     attention_mask=b_input_mask,
                     return_dict=True)
  logits = result.logits
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 2,100 test sentences...
    DONE.


In [25]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)

In [27]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
f1 = f1_score(flat_true_labels, flat_predictions)
accuracy = accuracy_score(flat_true_labels, flat_predictions)
precision = precision_score(flat_true_labels, flat_predictions)
recall = recall_score(flat_true_labels, flat_predictions)
conf_matrix = confusion_matrix(flat_true_labels, flat_predictions)

print('F1 Score: %.3f' % f1)
print('Accuracy: %.3f' % accuracy)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('Confusion Matrix:\n', conf_matrix)

F1 Score: 0.867
Accuracy: 0.864
Precision: 0.863
Recall: 0.870
Confusion Matrix:
 [[889 147]
 [138 926]]


In [28]:
import pickle

with open('final_model.pkl', 'wb') as file:
    pickle.dump(model, file)
